In [8]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io 
import random
import math
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
import seaborn as sn
import pandas as pd
os.environ['KMP_DUPLICATE_LIB_OK']='True' 

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import torch.optim as optim

Import Dataset

In [9]:
train_dl_origin = torch.load('Dataset/train_dl.pt')
valid_dl_origin = torch.load('Dataset/valid_dl.pt')

train_CSI = train_dl_origin.dataset[:][0]
train_label = train_dl_origin.dataset[:][1][:,2].type(torch.LongTensor)

valid_CSI = valid_dl_origin.dataset[:][0]
valid_label = valid_dl_origin.dataset[:][1][:,2].type(torch.LongTensor)

Seperate modulus and angle values, then combine into 1 dataset

In [10]:
train_a_raw = torch.angle(train_CSI).squeeze()
train_m_raw = torch.abs(train_CSI).squeeze()
valid_a_raw = torch.angle(valid_CSI).squeeze()
valid_m_raw = torch.abs(valid_CSI).squeeze()
train_am_raw = torch.stack((train_a_raw, train_m_raw), dim=3)
valid_am_raw = torch.stack((valid_a_raw, valid_m_raw), dim=3)

In [6]:
train_a_raw.shape

torch.Size([15000, 4, 1632])

Reshape to matrix

In [11]:
train_am_raw = train_am_raw.reshape((train_am_raw.shape[0], 4*1632*2))
valid_am_raw = valid_am_raw.reshape((valid_am_raw.shape[0], 4*1632*2))

Normalize to mean 0 std 1

In [15]:
mean = torch.mean(train_am_raw, dim=0)
std = torch.std(train_am_raw, dim=0)
train_am = (train_am_raw-mean)/(std)
valid_am = (valid_am_raw-mean)/(std)

Logistic Regression, accuracy of 100% on training, 90% on valid

In [259]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=10000)
lr.fit(train_am, train_label)

LogisticRegression(max_iter=10000)

In [263]:
accuracy_score(train_label, lr.predict(train_am)), accuracy_score(valid_label, lr.predict(valid_am))

(1.0, 0.8954)

SVM, accuracy of __ % on training, __ % on valid

In [17]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(train_am, train_label)

SVC()

In [18]:
accuracy_score(train_label, svc.predict(train_am)), accuracy_score(valid_label, svc.predict(valid_am))

(0.9862, 0.9472)

Did not try KNN due to high dimensionality of the data

Random forest using modulus and angle is much better, accuracy of 100% on training, 99.06% on valid

In [6]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(train_am, train_label)

RandomForestClassifier()

In [7]:
accuracy_score(train_label, rf.predict(train_am)), accuracy_score(valid_label, rf.predict(valid_am))

(1.0, 0.9906)

Neural Network to classify LoS and NLoS

In [77]:
class Model(nn.Module):
    def __init__(self, input_size):
        super(Model, self).__init__()
        
        self.fcs = nn.ModuleList([nn.Linear(input_size, 1000), nn.Linear(1000, 100)])
        self.output = nn.Linear(100, 1)
        
        self.relu = nn.ReLU()  
        self.dropout = nn.Dropout(.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        for fc in self.fcs:
            x = self.relu(self.dropout(fc(x)))
        x = self.output(x)
        x = self.dropout(x)
        x = self.sigmoid(x)
        return x.squeeze()
        
        
input_size = train_am.shape[1]
model = Model(input_size)

print(model)

Model(
  (fcs): ModuleList(
    (0): Linear(in_features=13056, out_features=1000, bias=True)
    (1): Linear(in_features=1000, out_features=100, bias=True)
  )
  (output): Linear(in_features=100, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)


Training

In [78]:
dataset = TensorDataset(train_am, train_label.to(torch.float32))
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

loss_fn = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
best_loss = 0.06520023941993713 # best loss achieved from previous training
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        
    # Check the validation loss
    model.eval()
    with torch.no_grad():
        outputs = model(valid_am)
        test_loss = loss_fn(outputs, valid_label.to(torch.float32))
        # do training loss as well due to dropout biasing the loss when in training mode
        outputs = model(train_am)
        loss = loss_fn(outputs, train_label.to(torch.float32))
    if (test_loss.item() < best_loss):
        best_loss = test_loss.item()
        torch.save(model.state_dict(), 'best-model-parameters-los.pt')
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item()}, Test Loss: {test_loss.item()}')

print('Training complete.')

Epoch [1/200], Training Loss: 0.4373367726802826, Test Loss: 0.44296348094940186
Epoch [2/200], Training Loss: 0.4260183870792389, Test Loss: 0.4327827990055084
Epoch [3/200], Training Loss: 0.4199770390987396, Test Loss: 0.4278047978878021
Epoch [4/200], Training Loss: 0.39976730942726135, Test Loss: 0.4104239344596863
Epoch [5/200], Training Loss: 0.38852372765541077, Test Loss: 0.401488721370697
Epoch [6/200], Training Loss: 0.3922261595726013, Test Loss: 0.4059831202030182
Epoch [7/200], Training Loss: 0.3840775787830353, Test Loss: 0.40156984329223633
Epoch [8/200], Training Loss: 0.3579406440258026, Test Loss: 0.37753522396087646
Epoch [9/200], Training Loss: 0.3286665976047516, Test Loss: 0.3558156490325928
Epoch [10/200], Training Loss: 0.37912267446517944, Test Loss: 0.39937520027160645
Epoch [11/200], Training Loss: 0.28174102306365967, Test Loss: 0.3193577229976654
Epoch [12/200], Training Loss: 0.3038058280944824, Test Loss: 0.33667659759521484
Epoch [13/200], Training Loss

In [80]:
model.eval()
train_out = model(train_am)
train_out = [1 if i > .5 else 0 for i in train_out]
outputs = model(valid_am)
outputs = [1 if i > .5 else 0 for i in outputs]
accuracy_score(train_label, train_out), accuracy_score(valid_label, outputs)

(1.0, 0.98)

Best model, 1 hidden layer, dropout(.5), ReLU and sigmoid, BCE Loss, lr=0.01

Training accuracy 100%, validation accuracy 98%

In [83]:
best = Model(input_size)
best.load_state_dict(torch.load("best-model-parameters-los.pt"))
best.eval()
train_out = model(train_am)
train_out = [1 if i > .5 else 0 for i in train_out]
outputs = model(valid_am)
test_loss = loss_fn(outputs, valid_label.to(torch.float32))
outputs = [1 if i > .5 else 0 for i in outputs]
accuracy_score(train_label, train_out), accuracy_score(valid_label, outputs)

(1.0, 0.98)